In [ ]:
!nvidia-smi

Sat Oct  7 08:17:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import time

import numpy as np
import pandas as pd
import imageio
#import imageio.v2 as imageio

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Define the paths to the training data
train_data_path = "/content/drive/MyDrive/WCEBleedGen/TrainData"
bleeding_path = train_data_path + "/bleeding"
non_bleeding_path = train_data_path + "/non-bleeding"
# Define the paths to the images and annotations
bleeding_images_path = bleeding_path + "/Images"
bleeding_annotations_path = bleeding_path + "/Annotations"
bleeding_bounding_boxes_path = bleeding_path + "/Bounding boxes/YOLO_TXT"
non_bleeding_images_path = non_bleeding_path + "/images"
non_bleeding_annotations_path = non_bleeding_path + "/annotation"

In [ ]:
# Bleeding
## Images
image_list_bleeding_orig = os.listdir(bleeding_images_path)
image_list_bleeding_orig = list(np.sort(image_list_bleeding_orig))
image_list_bleeding = [bleeding_images_path+'/'+i for i in image_list_bleeding_orig]
## Masks
mask_list_bleeding_orig = os.listdir(bleeding_annotations_path)
mask_list_bleeding_orig = list(np.sort(mask_list_bleeding_orig))
mask_list_bleeding = [bleeding_annotations_path+'/'+i for i in mask_list_bleeding_orig]

# Non Bleeding
## Images
image_list_non_bleeding_orig = os.listdir(non_bleeding_images_path)
image_list_non_bleeding_orig = list(np.sort(image_list_non_bleeding_orig))
image_list_non_bleeding = [non_bleeding_images_path+'/'+i for i in image_list_non_bleeding_orig]
## Masks
mask_list_non_bleeding_orig = os.listdir(non_bleeding_annotations_path)
mask_list_non_bleeding_orig = list(np.sort(mask_list_non_bleeding_orig))
mask_list_non_bleeding = [bleeding_annotations_path+'/'+i for i in mask_list_non_bleeding_orig]


Check some masked and unmasked images

In [ ]:
N = 23
img_t_Bld = imageio.imread(image_list_bleeding[N])
mask_t_Bld = imageio.imread(mask_list_bleeding[N])
# Convert the black and white mask to a binary mask
binary_mask = (mask_t_Bld > 0).astype(np.uint8)  # Assuming the object is represented by non-zero values

#mask = np.array([max(mask[i, j]) for i in range(mask.shape[0]) for j in range(mask.shape[1])]).reshape(img.shape[0], img.shape[1])

fig, arr = plt.subplots(1, 2, figsize=(14, 10))
arr[0].imshow(img_t_Bld)
arr[0].set_title('Image')
arr[1].imshow(binary_mask, cmap='gray')  # Use cmap='gray' to display it in grayscale
arr[1].set_title('Mask')

## Data Preprocess

In [ ]:
tf.constant(image_list_bleeding)

<tf.Tensor: shape=(1309,), dtype=string, numpy=
array([b'/content/drive/MyDrive/WCEBleedGen/TrainData/bleeding/Images/img- (1).png',
       b'/content/drive/MyDrive/WCEBleedGen/TrainData/bleeding/Images/img- (10).png',
       b'/content/drive/MyDrive/WCEBleedGen/TrainData/bleeding/Images/img- (100).png',
       ...,
       b'/content/drive/MyDrive/WCEBleedGen/TrainData/bleeding/Images/img- (997).png',
       b'/content/drive/MyDrive/WCEBleedGen/TrainData/bleeding/Images/img- (998).png',
       b'/content/drive/MyDrive/WCEBleedGen/TrainData/bleeding/Images/img- (999).png'],
      dtype=object)>

In [ ]:
def process_path(image_path, mask_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)
    mask = tf.math.reduce_max(mask, axis=-1, keepdims=True)
    return img, mask

def preprocess(image, mask):
    input_image = tf.image.resize(image, (96, 128), method='nearest')
    input_mask = tf.image.resize(mask, (96, 128), method='nearest')

    return input_image, input_mask

image_ds = dataset.map(process_path)
processed_image_ds = image_ds.map(preprocess)

In [ ]:
img = imageio.imread(image_list[N])
mask = imageio.imread(mask_list[N])
#mask = np.array([max(mask[i, j]) for i in range(mask.shape[0]) for j in range(mask.shape[1])]).reshape(img.shape[0], img.shape[1])

fig, arr = plt.subplots(1, 2, figsize=(14, 10))
arr[0].imshow(img)
arr[0].set_title('Image')
arr[1].imshow(mask[:, :, 0])
arr[1].set_title('Segmentation')

In [ ]:
# Create a list of all the image and annotation paths
image_paths = []
annotation_paths = []

In [ ]:
# Add the bleeding images and annotations to the lists
for i in range(1, 1310):
    image_path = bleeding_images_path + "/img- ({})".format(i) + ".png"
    annotation_path = bleeding_annotations_path + "/ann- ({})".format(i) + ".png"
    image_paths.append(image_path)
    annotation_paths.append(annotation_path)

In [ ]:
# Add the non-bleeding images and annotations to the lists
for i in range(1, 1310):
    image_path = non_bleeding_images_path + "/img- ({})".format(i) + ".png"
    annotation_path = non_bleeding_annotations_path + "/ann- ({})".format(i) + ".png"
    image_paths.append(image_path)
    annotation_paths.append(annotation_path)

In [ ]:
# Preprocess the images and annotations
def preprocess_image(image_path):
    """Preprocesses an image by converting it to TensorFlow format and resizing it to 224x224."""
    image = imageio.imread(image_path)
    image = image.astype("float32") / 255.0
    image = tf.image.resize(image, (224, 224))
    return image


In [ ]:
def preprocess_annotation(annotation_path):
    """Preprocesses an annotation by converting it to a TensorFlow format and resizing it to 224x224."""
    annotation = imageio.imread(annotation_path)
    annotation = annotation.astype("float32")
    annotation = tf.expand_dims(annotation, axis=0)
    annotation = tf.image.resize(annotation, (224, 224))
    return annotation

In [ ]:
# Preprocess the images and annotations
preprocessed_images = []
preprocessed_annotations = []

In [ ]:
def preprocess_dataset(start_indx, end_indx):
    for i in range(start_indx, end_indx):
        preprocessed_image = preprocess_image(image_paths[i])
        preprocessed_annotation = preprocess_annotation(annotation_paths[i])
        return preprocessed_image, preprocessed_annotation

In [ ]:
# Create a DataFrame from the generator
data_0 = pd.DataFrame(preprocess_dataset(0, 1309), columns=["image", "annotation"])
data_1 = pd.DataFrame(preprocess_dataset(1309, 2319), columns=["image", "annotation"])

In [ ]:
# Split the data into training and validation sets
train_data, val_data = tf.keras.utils.train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Save the training and validation sets to CSV files
train_data.to_csv("train_data.csv", index=False)
val_data.to_csv("val_data.csv", index=False)